In [28]:
from os import path
import imap_tools
from imap_tools import MailBox, AND
import pandas as pd
import datetime
from datetime import datetime as dt
from PIL import Image
import plotly.express as px
import config

In [29]:
camera_id = "Camera_trap_1"
mail_folder = camera_id+"/inbox"
mail_folder_treated = camera_id+"/treated"

In [30]:
data_path = "../data/"+camera_id

if not path.exists(data_path) :
    print("Project folder does not exist. Creating",data_path)
    os.mkdir(data_path)

In [31]:
def get_date_taken(path):
    return Image.open(path)._getexif()[36867]

In [32]:
# instantiate result dataframe
if not path.exists(data_path+"/database.csv") :
    print("Database file not found, creating one")
    results = pd.DataFrame()
else :
    print("Database file found, loading existing for completion")
    results = pd.read_csv(data_path+"/database.csv", index_col=[0])
    results["date_picture"] = results.apply(lambda x: dt.strptime(x["date_picture"], "%Y-%m-%d %H:%M:%S"), axis=1)

# connecting to mailbox
with MailBox(config.server).login(config.address, config.password, mail_folder) as mailbox:

    # we scan all messages on the mail_folder
    for msg in mailbox.fetch():
        
        # if there is more than one attached file, we raise an Exception
        if len(msg.attachments)>1:
            raise Exception("There is multiple attachments to this mail")

        # we scan each attached file of each mail
        for att in msg.attachments:
            
            # and extract information only when attached file has a .JPG extension
            if att.filename.endswith(".JPG"):
                
                # saving file
                try:
                    print(att.filename, att.content_type)
                    filename = data_path + '/' + str(msg.date.date()) + "_" + att.filename
                    with open(filename, 'wb') as f:
                        f.write(att.payload)
                except:
                    raise Warning("Something went wrong with file saving")

                # extracting metadata    
                try:
                    date_taken = dt.strptime(get_date_taken(filename), "%Y:%m:%d %H:%M:%S")
                    battery = float(msg.text.split("\r\n")[1].split(":")[1].replace("%",""))

                    # concatenating into output dataframe
                    results = pd.concat([results, pd.DataFrame({"subject":msg.subject, "text":msg.text, "date_picture":date_taken, "battery":battery, "filename":filename}, index=[0])])
                
                except:
                    print("Something went wrong with date taken or battery level extraction, did not save line")
                    pass
                    

    # flagging all messages of the folder as SEEN
    flags = (imap_tools.MailMessageFlags.SEEN)
    mailbox.flag(mailbox.uids(AND(seen=False)), flags, True)

    # MOVE all messages from current folder to INBOX/folder2
    mailbox.move(mailbox.uids(), mail_folder_treated)

results = results.reset_index(drop=True)
results.to_csv(data_path+"/database.csv")

Database file found, loading existing for completion


In [33]:
results

,subject,text,date_picture,battery,filename
0,BST886-4G-04/09 13:40-IMAG0174,[Instant]I got 44 pictures.Good luck! [44/100]...,2022-04-09 13:40:12,65.0,NaN
1,BST886-4G-04/09 13:35-IMAG0173,[Instant]I got 43 pictures.Good luck! [43/100]...,2022-04-09 13:35:12,65.0,NaN
2,BST886-4G-04/09 13:30-IMAG0172,[Instant]I got 42 pictures.Good luck! [42/100]...,2022-04-09 13:30:11,65.0,NaN
3,BST886-4G-04/09 13:25-IMAG0171,[Instant]I got 41 pictures.Good luck! [41/100]...,2022-04-09 13:25:14,65.0,NaN
4,BST886-4G-04/09 13:20-IMAG0170,[Instant]I got 40 pictures.Good luck! [40/100]...,2022-04-09 13:20:12,65.0,NaN
...,...,...,...,...,...
126,BST886-4G-01/01 00:25-IMAG0001,[Instant]I got a picture.Good luck! [1/100]\r\...,2021-01-01 00:25:16,92.0,NaN
127,BST886-4G-04/07 09:25-IMAG0004,[Instant]I got 2 pictures.Good luck! [2/100]\r...,2021-04-07 09:25:03,53.0,NaN
128,BST886-4G-04/07 09:17-IMAG0002,[Instant]I got a picture.Good luck! [1/100]\r\...,2021-04-07 09:16:59,52.0,NaN
129,BST886-4G-04/09 20:16-IMAG0175,[Instant]I got 45 pictures.Good luck! [45/100]...,2022-04-09 20:16:04,66.0,NaN


In [34]:
arr_date = results["date_picture"].dt.to_pydatetime()
results['date_picture']= pd.Series(arr_date, dtype=object)
results["battery"] = results.apply(lambda x: float(x["battery"]), axis=1)

In [35]:
results = results[results["date_picture"] > datetime.datetime(2022,1,1,0,0,0)]

In [36]:
fig = px.scatter(results, x="date_picture", y="battery", trendline="rolling", trendline_options=dict(window=5), title="battery level (%)")
fig.show()